## 버스 정류장
url="http://openapi.seoul.go.kr:8088/{key}/xml/CardBusStatisticsServiceNew/{startIndex}/{endIndex}/20151101/ "

## 교통 지점 데이터
url="http://openapi.seoul.go.kr:8088/{key}/xml/SpotInfo/{startIndex}/{endIndex}/"

## 지하철 역 정보
url="http://openAPI.seoul.go.kr:8088/{key}/xml/SearchLocationOfSTNByIDService/{startIndex}/{endIndex}/{stationNumber}/"

## 지하철 승하차 인원
urlSub="http://openapi.seoul.go.kr:8088/{key}/xml/CardSubwayStatsNew/{startIndex}/{endIndex}/{date}" 

In [60]:
import urllib.request
from lxml import etree
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
from xml.etree.ElementTree import parse
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
serviceKey="416b486a496b6861373365594b4642"
serviceKey2="704a534e596b686137354d61686e42"
urlSub="http://openapi.seoul.go.kr:8088/{key}/xml/CardSubwayStatsNew/{startIndex}/{endIndex}/{date}"

In [3]:
urlSub

'http://openapi.seoul.go.kr:8088/{key}/xml/CardSubwayStatsNew/{startIndex}/{endIndex}/{date}'

In [4]:
urllib.request.urlopen(urlSub.format(key=serviceKey, startIndex=1, endIndex=1000,date="20180101"))

In [5]:
print(urlSub.format(key=serviceKey2, startIndex=1, endIndex=1000, date="20180101"))

http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardSubwayStatsNew/1/1000/20180101


In [6]:
print(urlSub.format(key=serviceKey2, startIndex=1, endIndex=1000, date="20180102"))

http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardSubwayStatsNew/1/1000/20180102


In [7]:
u=urlopen(urlSub.format(key=serviceKey, startIndex=1, endIndex=1000,date="20180101"))
doc=parse(u)

# Extract and output tags of interest

In [8]:
# USE_DT=[]
# LINE_NUM=[]
# SUB_STA_NM=[]
# RIDE_PASGR_NUM=[]
# ALIGHT_PASGR_NUM=[]

In [9]:
def dataScrapping(sdate):
    url=urlSub.format(key=serviceKey2, startIndex=1, endIndex=1000,date=sdate)
    req=requests.get(url)
    html=req.text
    #print(html)
    soup=BeautifulSoup(html,'html.parser')

    #print(soup)
    USE_DT=pd.Series(soup.find_all('use_dt'))
    LINE_NUM=pd.Series(soup.find_all('line_num'))
    SUB_STA_NM=pd.Series(soup.find_all('sub_sta_nm'))
    RIDE_PASGR_NUM=pd.Series(soup.find_all('ride_pasgr_num'))
    ALIGHT_PASGR_NUM=pd.Series(soup.find_all('alight_pasgr_num'))

    idx=pd.Series(list(range(len(USE_DT))))
    
# Data Parsing
        ## 탑승일
    USE_DT=list(soup.find_all('use_dt'))
    temp=[]
    for da in USE_DT:
        temp.append((str(da).split("<use_dt>"))[1].split("</use_dt>")[0])
    USE_DT=pd.Series(temp)

    ## 탑승 호선
    LINE_NUM=list(soup.find_all('line_num'))
    temp=[]
    for da in LINE_NUM:
        temp.append((str(da).split("<line_num>"))[1].split("</line_num>")[0])

    LINE_NUM=pd.Series(temp)

    ## 탑승역
    SUB_STA_NM=list(soup.find_all('sub_sta_nm'))
    temp=[]
    for da in SUB_STA_NM:
        temp.append((str(da).split("<sub_sta_nm>"))[1].split("</sub_sta_nm>")[0])

    SUB_STA_NM=pd.Series(temp)

    ## 탑승인원
    RIDE_PASGR_NUM=list(soup.find_all('ride_pasgr_num'))
    temp=[]
    for da in RIDE_PASGR_NUM:
        temp.append((str(da).split("<ride_pasgr_num>"))[1].split("</ride_pasgr_num>")[0])

    RIDE_PASGR_NUM=pd.Series(temp)

    ## 하차인원
    ALIGHT_PASGR_NUM=list(soup.find_all('alight_pasgr_num'))
    temp=[]
    for da in ALIGHT_PASGR_NUM:
        temp.append((str(da).split("<alight_pasgr_num>"))[1].split("</alight_pasgr_num>")[0])

    ALIGHT_PASGR_NUM=pd.Series(temp)

    
# Make DataFrame    
    USE_DT=pd.DataFrame(data={'use_dt':USE_DT, 'index': idx},index=range(len(USE_DT)))
    LINE_NUM=pd.DataFrame(data={'line_num':LINE_NUM, 'index':idx}, index=range(len(LINE_NUM)))
    SUB_STA_NM=pd.DataFrame(data={'sub_sta_nm':SUB_STA_NM,'index':idx}, index=range(len(SUB_STA_NM)))
    RIDE_PASGR_NUM=pd.DataFrame(data={'ride_pasgr_num':RIDE_PASGR_NUM,'index':idx}, index=range(len(RIDE_PASGR_NUM)))
    ALIGHT_PASGR_NUM=pd.DataFrame(data={'alight_pasgr_num':ALIGHT_PASGR_NUM,'index':idx}, index=range(len(ALIGHT_PASGR_NUM)))
    
    
# Mergind Data
    df=pd.merge(USE_DT, LINE_NUM, how='left')
    df=pd.merge(df,SUB_STA_NM, how='left')
    df=pd.merge(df,RIDE_PASGR_NUM,how='left')
    df=pd.merge(df,ALIGHT_PASGR_NUM,how='left')

    df.drop('index',axis=1, inplace=True)
    return df

In [10]:
#while endnumber<=1000:
url=urlSub.format(key=serviceKey2, startIndex=1, endIndex=1000,date="20180101")
req=requests.get(url)
html=req.text
#print(html)
soup=BeautifulSoup(html,'html.parser')

#print(soup)
USE_DT=pd.Series(soup.find_all('use_dt'))
LINE_NUM=pd.Series(soup.find_all('line_num'))
SUB_STA_NM=pd.Series(soup.find_all('sub_sta_nm'))
RIDE_PASGR_NUM=pd.Series(soup.find_all('ride_pasgr_num'))
ALIGHT_PASGR_NUM=pd.Series(soup.find_all('alight_pasgr_num'))

idx=pd.Series(list(range(len(USE_DT))))


In [11]:
## 탑승일
USE_DT=list(soup.find_all('use_dt'))
temp=[]
for da in USE_DT:
    temp.append((str(da).split("<use_dt>"))[1].split("</use_dt>")[0])
USE_DT=pd.Series(temp)

## 탑승 호선
LINE_NUM=list(soup.find_all('line_num'))
temp=[]
for da in LINE_NUM:
    temp.append((str(da).split("<line_num>"))[1].split("</line_num>")[0])
    
LINE_NUM=pd.Series(temp)

## 탑승역
SUB_STA_NM=list(soup.find_all('sub_sta_nm'))
temp=[]
for da in SUB_STA_NM:
    temp.append((str(da).split("<sub_sta_nm>"))[1].split("</sub_sta_nm>")[0])
    
SUB_STA_NM=pd.Series(temp)

## 탑승인원
RIDE_PASGR_NUM=list(soup.find_all('ride_pasgr_num'))
temp=[]
for da in RIDE_PASGR_NUM:
    temp.append((str(da).split("<ride_pasgr_num>"))[1].split("</ride_pasgr_num>")[0])
    
RIDE_PASGR_NUM=pd.Series(temp)

## 하차인원
ALIGHT_PASGR_NUM=list(soup.find_all('alight_pasgr_num'))
temp=[]
for da in ALIGHT_PASGR_NUM:
    temp.append((str(da).split("<alight_pasgr_num>"))[1].split("</alight_pasgr_num>")[0])
    
ALIGHT_PASGR_NUM=pd.Series(temp)



In [12]:
columns=['USE_DT','LINE_NUM','SUB_STA_NM','RIDE_PASGR_NUM','ALIGHT_PASGR_NUM']

In [13]:
USE_DT=pd.DataFrame(data={'use_dt':USE_DT, 'index': idx},index=range(len(USE_DT)))
LINE_NUM=pd.DataFrame(data={'line_num':LINE_NUM, 'index':idx}, index=range(len(LINE_NUM)))
SUB_STA_NM=pd.DataFrame(data={'sub_sta_nm':SUB_STA_NM,'index':idx}, index=range(len(SUB_STA_NM)))
RIDE_PASGR_NUM=pd.DataFrame(data={'ride_pasgr_num':RIDE_PASGR_NUM,'index':idx}, index=range(len(RIDE_PASGR_NUM)))
ALIGHT_PASGR_NUM=pd.DataFrame(data={'alight_pasgr_num':ALIGHT_PASGR_NUM,'index':idx}, index=range(len(ALIGHT_PASGR_NUM)))

In [14]:
print(USE_DT.shape)
print(LINE_NUM.shape)
print(SUB_STA_NM.shape)
print(RIDE_PASGR_NUM.shape)
print(ALIGHT_PASGR_NUM.shape)

(579, 2)
(579, 2)
(579, 2)
(579, 2)
(579, 2)


In [15]:
USE_DT.head()

,use_dt,index
0,20180101,0
1,20180101,1
2,20180101,2
3,20180101,3
4,20180101,4


## Merging data

In [16]:
df=pd.merge(USE_DT, LINE_NUM, how='left')
df=pd.merge(df,SUB_STA_NM, how='left')
df=pd.merge(df,RIDE_PASGR_NUM,how='left')
df=pd.merge(df,ALIGHT_PASGR_NUM,how='left')

df.drop('index',axis=1, inplace=True)
df


,use_dt,line_num,sub_sta_nm,ride_pasgr_num,alight_pasgr_num
0,20180101,1호선,서울역,40213,34505
1,20180101,1호선,시청,12256,10889
2,20180101,1호선,종각,22524,17721
3,20180101,1호선,종로3가,18578,15828
4,20180101,1호선,종로5가,10588,10225
5,20180101,1호선,동대문,10044,10855
6,20180101,1호선,신설동,7257,7050
7,20180101,1호선,제기동,10290,10559
8,20180101,1호선,청량리(서울시립대입구),17025,17058
9,20180101,1호선,동묘앞,7858,8346


In [17]:
howmanydays=[31,28,31,30,31,30,31,31,30,31,30,31]
month=["01","02","03","04","05","06","07","08","09","10","11","12"]
stdString="2018"
date=[]
for i in range(1, 13, 1):
    for j in range(1, howmanydays[i-1]+1, 1):
        if j<10 : 
            date.append(stdString+month[i-1]+"0"+str(j))
        else:
            date.append(stdString+month[i-1]+str(j))

In [18]:
date

['20180101',
 '20180102',
 '20180103',
 '20180104',
 '20180105',
 '20180106',
 '20180107',
 '20180108',
 '20180109',
 '20180110',
 '20180111',
 '20180112',
 '20180113',
 '20180114',
 '20180115',
 '20180116',
 '20180117',
 '20180118',
 '20180119',
 '20180120',
 '20180121',
 '20180122',
 '20180123',
 '20180124',
 '20180125',
 '20180126',
 '20180127',
 '20180128',
 '20180129',
 '20180130',
 '20180131',
 '20180201',
 '20180202',
 '20180203',
 '20180204',
 '20180205',
 '20180206',
 '20180207',
 '20180208',
 '20180209',
 '20180210',
 '20180211',
 '20180212',
 '20180213',
 '20180214',
 '20180215',
 '20180216',
 '20180217',
 '20180218',
 '20180219',
 '20180220',
 '20180221',
 '20180222',
 '20180223',
 '20180224',
 '20180225',
 '20180226',
 '20180227',
 '20180228',
 '20180301',
 '20180302',
 '20180303',
 '20180304',
 '20180305',
 '20180306',
 '20180307',
 '20180308',
 '20180309',
 '20180310',
 '20180311',
 '20180312',
 '20180313',
 '20180314',
 '20180315',
 '20180316',
 '20180317',
 '20180318',

In [19]:
data=dataScrapping(date[0]) # 조회 시작일 2018년 1월 1일
for d in date[1:]:
    #print(d)
    temp=dataScrapping(d)
    #print(temp.shape)
    data=pd.concat([data,temp])
    
data.shape

(212157, 5)

In [20]:
data.sample(10)

,use_dt,line_num,sub_sta_nm,ride_pasgr_num,alight_pasgr_num
104,20180506,4호선,한성대입구(삼선교),10022,9530
213,20180826,7호선,중계,9466,8776
358,20181130,경원선,한남,5025,4961
495,20180531,분당선,구성,5957,5911
309,20181120,경강선,삼동,2570,2190
415,20181117,경인선,부천,40443,42190
567,20180131,중앙선,양정,938,837
221,20181013,7호선,사가정,15405,14543
76,20180219,3호선,옥수,10223,10914
338,20180224,경부선,독산,11321,11465


In [21]:
data.to_csv("지하철승하차.csv", encoding="utf-8")

In [22]:
data[['ride_pasgr_num','alight_pasgr_num']]=data[['ride_pasgr_num','alight_pasgr_num']].astype(int)

In [23]:
data.groupby('sub_sta_nm')[['ride_pasgr_num','alight_pasgr_num']].sum()

,ride_pasgr_num,alight_pasgr_num
sub_sta_nm,,
4.19민주묘지,1104153,1084521
가능,2670280,2503298
가락시장,6392767,6719133
가산디지털단지,19713310,20648602
가양,7463286,7159536
가오리,1415158,1381317
가좌,1875542,1724181
가천대,3415606,3808975
가평,896835,934233


In [24]:
urlStation="http://openAPI.seoul.go.kr:8088/{key}/xml/SearchInfoBySubwayNameService/{startIndex}/{endIndex}/{station_name}/"
def dataScrappingStation(sName):
    url=urlStation.format(key=serviceKey2, startIndex=1, endIndex=1000,station_name=sName)
    print(url)
    req=requests.get(url)
    html=req.text
    #print(html)
    soup=BeautifulSoup(html,'html.parser')

    #(soup)
    STAION_CD=pd.Series(soup.find_all('station_cd'))
    STATION_NM=pd.Series(soup.find_all('station_nm'))
    LINE_NUM=pd.Series(soup.find_all('line_num'))
    FR_CODE=pd.Series(soup.find_all('fr_code'))
    
    idx=pd.Series(list(range(len(STAION_CD))))
    
# Data Parsing
    ## 역 코드
    STAION_CD=list(soup.find_all('station_cd'))
    temp=[]
    for da in STAION_CD:
        temp.append((str(da).split("<station_cd>"))[1].split("</station_cd>")[0])
    STAION_CD=pd.Series(temp)

    ## 역 이름
    STATION_NM=list(soup.find_all('station_nm'))
    temp=[]
    for da in STATION_NM:
        temp.append((str(da).split("<station_nm>"))[1].split("</station_nm>")[0])

    STATION_NM=pd.Series(temp)

    ## 호선
    LINE_NUM=list(soup.find_all('line_num'))
    temp=[]
    for da in LINE_NUM:
        temp.append((str(da).split("<line_num>"))[1].split("</line_num>")[0])

    LINE_NUM=pd.Series(temp)

    ## 외부코드
    FR_CODE=list(soup.find_all('fr_code'))
    temp=[]
    for da in FR_CODE:
        temp.append((str(da).split("<fr_code>"))[1].split("</fr_code>")[0])

    FR_CODE=pd.Series(temp)

    
# Make DataFrame    
    STAION_CD=pd.DataFrame(data={'station_cd':STAION_CD, 'index': idx},index=range(len(STAION_CD)))
    STATION_NM=pd.DataFrame(data={'station_nm':STATION_NM, 'index':idx}, index=range(len(STATION_NM)))
    LINE_NUM=pd.DataFrame(data={'line_num':LINE_NUM,'index':idx}, index=range(len(LINE_NUM)))
    FR_CODE=pd.DataFrame(data={'fr_code':FR_CODE,'index':idx}, index=range(len(FR_CODE)))
    
    
# Mergind Data
    df=pd.merge(STAION_CD, STATION_NM, how='left')
    df=pd.merge(df,LINE_NUM, how='left')
    df=pd.merge(df,FR_CODE,how='left')

    df.drop('index',axis=1, inplace=True)
    return df

In [25]:
stationName=data['sub_sta_nm'].unique()

In [26]:
stationName=list(stationName)
len(stationName)

509

In [27]:
stationData=dataScrapping(stationName[0]) # 조회 시작일 2018년 1월 1일
for d in stationName[1:]:
    #print(d)
    temp=dataScrappingStation(d)
    print(temp)
    if temp.shape[0]==0:
        continue
    stationData=pd.concat([stationData,temp])
    
stationData.shape

http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/시청/
  station_cd station_nm line_num fr_code
0       0201         시청     02호선     201
1       0151         시청     01호선     132
http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/종각/
  station_cd station_nm line_num fr_code
0       0152         종각     01호선     131
http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/종로3가/
  station_cd station_nm line_num fr_code
0       0319       종로3가     03호선     329
1       2535       종로3가     05호선     534
2       0153       종로3가     01호선     130
http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/종로5가/


C:\Users\B-17\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


  station_cd station_nm line_num fr_code
0       0154       종로5가     01호선     129
http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/동대문/
  station_cd station_nm line_num fr_code
0       0421        동대문     04호선     421
1       0155        동대문     01호선     128
http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/신설동/
  station_cd station_nm line_num fr_code
0       0246        신설동     02호선   211-4
1       4713        신설동  우이신설경전철     953
2       0156        신설동     01호선     126
http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/제기동/
  station_cd station_nm line_num fr_code
0       0157        제기동     01호선     125
http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/청량리(서울시립대입구)/
Empty DataFrame
Columns: [station_cd, station_nm, line_num, fr_code]
Index: []
http://openAPI.seoul.go.k

  station_cd station_nm line_num fr_code
0       0236      영등포구청     02호선     236
1       2524      영등포구청     05호선     523
http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/당산/
  station_cd station_nm line_num fr_code
0       0237         당산     02호선     237
1       4113         당산     09호선     913
http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/합정/
  station_cd station_nm line_num fr_code
0       0238         합정     02호선     238
1       2623         합정     06호선     622
http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/홍대입구/
  station_cd station_nm line_num fr_code
0       0239       홍대입구     02호선     239
1       1264       홍대입구      경의선    K314
2       4203       홍대입구     공항철도     A03
http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/신촌/
  station_cd station_nm line_num 

  station_cd station_nm line_num fr_code
0       0342         오금     03호선     352
1       2558         오금     05호선    P552
http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/당고개/
  station_cd station_nm line_num fr_code
0       0409        당고개     04호선     409
http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/상계/
  station_cd station_nm line_num fr_code
0       0410         상계     04호선     410
http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/노원/
  station_cd station_nm line_num fr_code
0       0411         노원     04호선     411
1       2715         노원     07호선     713
http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/창동/
  station_cd station_nm line_num fr_code
0       0412         창동     04호선     412
1       1022         창동     01호선     116
http://openAPI.seoul.go.kr:8088/70

...

  station_cd station_nm line_num fr_code
0       1280         금촌      경의선    K331
http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/월롱/
  station_cd station_nm line_num fr_code
0       1282         월롱      경의선    K333
http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/파주/
  station_cd station_nm line_num fr_code
0       1283         파주      경의선    K334
http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/문산/
  station_cd station_nm line_num fr_code
0       1284         문산      경의선    K335
http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/개봉/
  station_cd station_nm line_num fr_code
0       1801         개봉     01호선     143
http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/오류동/
  station_cd station_nm line_num fr_code
0       

http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/검암/
  station_cd station_nm line_num fr_code
0       3207         검암    인천2호선    I207
1       4209         검암     공항철도     A07
http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/청라국제도시/
  station_cd station_nm line_num fr_code
0       4210     청라국제도시     공항철도    A071
http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/운서/
  station_cd station_nm line_num fr_code
0       4211         운서     공항철도     A08
http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/공항화물청사/
  station_cd station_nm line_num fr_code
0       4212     공항화물청사     공항철도     A09
http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/인천공항1터미널/
  station_cd station_nm line_num fr_code
0       4213   인천공항1터미널     공항철도    

  station_cd station_nm line_num fr_code
0       1883    남동인더스파크      수인선    K256
http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/원인재/
  station_cd station_nm line_num fr_code
0       1884        원인재      수인선    K257
1       3130        원인재      인천선    I130
http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/연수/
  station_cd station_nm line_num fr_code
0       1885         연수      수인선    K258
http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/송도/
  station_cd station_nm line_num fr_code
0       1886         송도      수인선    K259
http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/인하대/
  station_cd station_nm line_num fr_code
0       1888        인하대      수인선    K261
http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/숭의/
  stati

  station_cd station_nm line_num fr_code
0       1405         배방     01호선    P174
http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/온양온천/
  station_cd station_nm line_num fr_code
0       1407       온양온천     01호선    P176
http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/신창(순천향대)/
Empty DataFrame
Columns: [station_cd, station_nm, line_num, fr_code]
Index: []
http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/회기/
  station_cd station_nm line_num fr_code
0       1015         회기     01호선     123
1       1307         회기      경춘선    P118
2       101C         회기      경의선    K118
http://openAPI.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/SearchInfoBySubwayNameService/1/1000/중랑/
  station_cd station_nm line_num fr_code
0       1201         중랑      경의선    K119
1       1308         중랑      경춘선    P119
http://openAPI.seoul.go.kr:808

(541, 8)

In [28]:
stationData=stationData[['station_cd','fr_code','station_nm','line_num']]

In [29]:
stationData.to_csv("지하철역 코드 정보.csv", encoding="utf-8");

In [30]:
stationData.groupby(['line_num','station_nm']).sum()
stationData['line_num'].value_counts()

01호선       93
경의선        50
07호선       44
02호선       41
05호선       40
03호선       40
04호선       38
분당선        35
09호선       35
06호선       26
경춘선        23
수인선        14
08호선       13
공항철도       13
경강선        11
우이신설경전철    11
인천선         4
신분당선        4
서해선         2
인천2호선       2
용인경전철       1
의정부경전철      1
Name: line_num, dtype: int64

    1   USE_DT        	 사용일자
    2	BUS_ROUTE_ID	 노선ID
    3	BUS_ROUTE_NO	 노선번호
    4	BUS_ROUTE_NM	 노선명
    5	STND_BSST_ID	 표준버스정류장ID
    7	BUS_STA_ID	     역ID
    8	BUS_STA_NM	     역명
    9	RIDE_PASGR_NUM	 승차총승객수
    10	ALIGHT_PASGR_NUM 하차총승객수

In [31]:
busUserUrl="http://openapi.seoul.go.kr:8088/{key}/xml/CardBusStatisticsServiceNew/{startIndex}/{endIndex}/{date}/ "
def busUserInfo(inputDate):
    url=busUserUrl.format(key=serviceKey2, startIndex=1, endIndex=1000,date=inputDate)
    print(url)
    req=requests.get(url)
    html=req.text
    #print(html)
    soup=BeautifulSoup(html,'html.parser')

# #Find Data
#     USE_DT=pd.Series(soup.find_all('use_dt'))
#     BUS_ROUTE_ID=pd.Series(soup.find_all('bus_route_id'))
#     BUS_ROUTE_NO=pd.Series(soup.find_all('bus_route_no'))
#     BUS_ROUTE_NM=pd.Series(soup.find_all('bus_route_nm'))
#     STND_BSST_ID=pd.Series(soup.find_all('stnd_bsst_id'))
#     BUS_STA_ID=pd.Series(soup.find_all('bus_sta'))
#     BUS_STA_NM=pd.Series(soup.find_all('bus_sta_nm'))
#     RIDE_PASGR_NUM=pd.Series(soup.find_all('ride_pasgr_num'))
#     ALIGHT_PASGR_NUM=pd.Series(soup.find_all('alight_pasgr_num'))

    
# Data Parsing
    ## 사용 일자
    USE_DT=list(soup.find_all('use_dt'))
    temp=[]
    for da in USE_DT:
        temp.append((str(da).split("<use_dt>"))[1].split("</use_dt>")[0])
    USE_DT=pd.Series(temp)

    ## 노선 ID
    BUS_ROUTE_ID=list(soup.find_all('bus_route_id'))
    temp=[]
    for da in BUS_ROUTE_ID:
        temp.append((str(da).split("<bus_route_id>"))[1].split("</bus_route_id>")[0])

    BUS_ROUTE_ID=pd.Series(temp)

    ## 노선번호
    BUS_ROUTE_NO=list(soup.find_all('bus_route_no'))
    temp=[]
    for da in BUS_ROUTE_NO:
        temp.append((str(da).split("<bus_route_no>"))[1].split("</bus_route_no>")[0])

    BUS_ROUTE_NO=pd.Series(temp)
    

    ## 노선명
    BUS_ROUTE_NM=list(soup.find_all('bus_route_nm'))
    temp=[]
    for da in BUS_ROUTE_NM:
        temp.append((str(da).split("<bus_route_nm>"))[1].split("</bus_route_nm>")[0])

    BUS_ROUTE_NM=pd.Series(temp)

      ##  표준버스정류장ID
    STND_BSST_ID=list(soup.find_all('stnd_bsst_id'))
    temp=[]
    for da in STND_BSST_ID:
        temp.append((str(da).split("<stnd_bsst_id>"))[1].split("</stnd_bsst_id>")[0])

    STND_BSST_ID=pd.Series(temp)

      ## 역ID
    BUS_STA_ID=list(soup.find_all('bus_sta_id'))
    temp=[]
    for da in BUS_STA_ID:
        temp.append((str(da).split("<bus_sta_id>"))[1].split("</bus_sta_id>")[0])

    BUS_STA_ID=pd.Series(temp)

      ## 역명
    BUS_STA_NM=list(soup.find_all('bus_sta_nm'))
    temp=[]
    for da in BUS_STA_NM:
        temp.append((str(da).split("<bus_sta_nm>"))[1].split("</bus_sta_nm>")[0])

    BUS_STA_NM=pd.Series(temp)

      ## 승차총승객수
    RIDE_PASGR_NUM=list(soup.find_all('ride_pasgr_num'))
    temp=[]
    for da in RIDE_PASGR_NUM:
        temp.append((str(da).split("<ride_pasgr_num>"))[1].split("</ride_pasgr_num>")[0])

    RIDE_PASGR_NUM=pd.Series(temp)

      ##  하차총승객수
    ALIGHT_PASGR_NUM=list(soup.find_all('alight_pasgr_num'))
    temp=[]
    for da in ALIGHT_PASGR_NUM:
        temp.append((str(da).split("<alight_pasgr_num>"))[1].split("</alight_pasgr_num>")[0])

    ALIGHT_PASGR_NUM=pd.Series(temp)
    
# Make Index    

    idx=pd.Series(list(range(len(USE_DT))))

# Make DataFrame    
    USE_DT=pd.DataFrame(data={'use_dt':USE_DT, 'index': idx},index=range(len(USE_DT)))
    BUS_ROUTE_ID=pd.DataFrame(data={'bus_route_id':BUS_ROUTE_ID, 'index':idx}, index=range(len(BUS_ROUTE_ID)))
    BUS_ROUTE_NO=pd.DataFrame(data={'bus_route_no':BUS_ROUTE_NO,'index':idx}, index=range(len(BUS_ROUTE_NO)))
    BUS_ROUTE_NM=pd.DataFrame(data={'bus_route_nm':BUS_ROUTE_NM,'index':idx}, index=range(len(BUS_ROUTE_NM)))
    STND_BSST_ID=pd.DataFrame(data={'stnd_bsst_id':STND_BSST_ID,'index':idx}, index=range(len(STND_BSST_ID)))
    BUS_STA_ID=pd.DataFrame(data={'bus_sta_id':BUS_STA_ID,'index':idx}, index=range(len(BUS_STA_ID)))
    BUS_STA_NM=pd.DataFrame(data={'bus_sta_nm':BUS_STA_NM,'index':idx}, index=range(len(BUS_STA_NM)))
    RIDE_PASGR_NUM=pd.DataFrame(data={'ride_pasgr_num':RIDE_PASGR_NUM,'index':idx}, index=range(len(RIDE_PASGR_NUM)))
    ALIGHT_PASGR_NUM=pd.DataFrame(data={'alight_pasgr_num':ALIGHT_PASGR_NUM,'index':idx}, index=range(len(ALIGHT_PASGR_NUM)))


    
# Mergind Data
    df=pd.merge(USE_DT, BUS_ROUTE_ID, how='left')
    df=pd.merge(df,BUS_ROUTE_NO, how='left')
    df=pd.merge(df,BUS_ROUTE_NM,how='left')
    df=pd.merge(df,STND_BSST_ID,how='left')
    df=pd.merge(df,BUS_ROUTE_NM,how='left')
    df=pd.merge(df,BUS_STA_ID,how='left')
    df=pd.merge(df,BUS_STA_NM,how='left')
    df=pd.merge(df,RIDE_PASGR_NUM,how='left')
    df=pd.merge(df,ALIGHT_PASGR_NUM,how='left')

    df.drop('index',axis=1, inplace=True)
    return df

In [32]:
howmanydays=[31,28,31,30,31,30,31,31,30,31,30,31]
month=["01","02","03","04","05","06","07","08","09","10","11","12"]
stdString="2018"
date=[]
for i in range(1, 13, 1):
    for j in range(1, howmanydays[i-1]+1, 1):
        if j<10 : 
            date.append(stdString+month[i-1]+"0"+str(j))
        else:
            date.append(stdString+month[i-1]+str(j))
date

['20180101',
 '20180102',
 '20180103',
 '20180104',
 '20180105',
 '20180106',
 '20180107',
 '20180108',
 '20180109',
 '20180110',
 '20180111',
 '20180112',
 '20180113',
 '20180114',
 '20180115',
 '20180116',
 '20180117',
 '20180118',
 '20180119',
 '20180120',
 '20180121',
 '20180122',
 '20180123',
 '20180124',
 '20180125',
 '20180126',
 '20180127',
 '20180128',
 '20180129',
 '20180130',
 '20180131',
 '20180201',
 '20180202',
 '20180203',
 '20180204',
 '20180205',
 '20180206',
 '20180207',
 '20180208',
 '20180209',
 '20180210',
 '20180211',
 '20180212',
 '20180213',
 '20180214',
 '20180215',
 '20180216',
 '20180217',
 '20180218',
 '20180219',
 '20180220',
 '20180221',
 '20180222',
 '20180223',
 '20180224',
 '20180225',
 '20180226',
 '20180227',
 '20180228',
 '20180301',
 '20180302',
 '20180303',
 '20180304',
 '20180305',
 '20180306',
 '20180307',
 '20180308',
 '20180309',
 '20180310',
 '20180311',
 '20180312',
 '20180313',
 '20180314',
 '20180315',
 '20180316',
 '20180317',
 '20180318',

In [33]:
# stationData=busUserInfo(stationName[0]) # 조회 시작일 2018년 1월 1일
# for d in stationName[1:]:
#     #print(d)
#     temp=busUserInfo(d)
#     print(temp)
#     if temp.shape[0]==0:
#         continue
#     stationData=pd.concat([stationData,temp])
    
# stationData.shape


data=busUserInfo(date[0]) # 조회 시작일 2018년 1월 1일
for d in date[1:]:
    #print(d)
    temp=busUserInfo(d)
    #print(temp.shape)
    data=pd.concat([data,temp])
    
data.shape

http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180101/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180102/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180103/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180104/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180105/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180106/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180107/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180108/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1

http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180315/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180316/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180317/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180318/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180319/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180320/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180321/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180322/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1

http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180527/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180528/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180529/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180530/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180531/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180601/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180602/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180603/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1

C:\Users\B-17\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180616/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180617/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180618/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180619/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180620/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180621/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180622/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180623/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1

http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180828/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180829/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180830/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180831/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180901/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180902/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180903/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20180904/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1

http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20181109/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20181110/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20181111/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20181112/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20181113/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20181114/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20181115/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1/1000/20181116/ 
http://openapi.seoul.go.kr:8088/704a534e596b686137354d61686e42/xml/CardBusStatisticsServiceNew/1

(361024, 9)

In [46]:
data.head()

,alight_pasgr_num,bus_route_id,bus_route_nm,bus_route_no,bus_sta_id,bus_sta_nm,ride_pasgr_num,stnd_bsst_id,use_dt
0,114,11110001,100번(하계동~용산구청),100,0007197,명륜3가.성대입구,74,100000003,20180101
1,31,11110001,100번(하계동~용산구청),100,0008839,을지로입구.로얄호텔,105,101000057,20180101
2,27,11110001,100번(하계동~용산구청),100,0008864,을지로2가.파인에빈뉴,57,101000060,20180101
3,7,11110001,100번(하계동~용산구청),100,0008878,서빙고역교차로,8,102000173,20180101
4,83,11110001,100번(하계동~용산구청),100,0008881,을지로3가,20,101000061,20180101


In [35]:
data.to_csv("버스 승하차인원데이터.csv", encoding="utf-8")

In [38]:
busUser=pd.read_csv("버스 승하차인원데이터.csv", encoding="utf-8")


C:\Users\B-17\AppData\Roaming\Python\Python36\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,Unnamed: 0,alight_pasgr_num,bus_route_id,bus_route_nm,bus_route_no,bus_sta_id,bus_sta_nm,ride_pasgr_num,stnd_bsst_id,use_dt


In [39]:
busUser.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361024 entries, 0 to 361023
Data columns (total 10 columns):
Unnamed: 0          361024 non-null int64
alight_pasgr_num    361024 non-null int64
bus_route_id        361024 non-null int64
bus_route_nm        361024 non-null object
bus_route_no        361024 non-null object
bus_sta_id          361024 non-null int64
bus_sta_nm          361024 non-null object
ride_pasgr_num      361024 non-null int64
stnd_bsst_id        361024 non-null int64
use_dt              361024 non-null int64
dtypes: int64(7), object(3)
memory usage: 27.5+ MB


In [51]:
busUser.sample(3)

,Unnamed: 0,alight_pasgr_num,bus_route_id,bus_route_nm,bus_route_no,bus_sta_id,bus_sta_nm,ride_pasgr_num,stnd_bsst_id,use_dt
39899,899,271,11110013,142번(도봉동~고속터미널),142,12869,월곡뉴타운,153,107000002,20180209
132813,789,9,11110012,141번(도봉산~염곡동),141,70462,개포주공1단지아파트,81,122000345,20180514
234784,760,292,11110011,140번(도봉산역~AT센터),140,9006330,양재역.서초문화예술회관,236,121000003,20180827


In [ ]:
plt.

In [59]:
busUser[busUser['bus_sta_nm']=="월곡뉴타운"].groupby("use_dt")['alight_pasgr_num'].sum()

use_dt
20180101     721
20180102    1664
20180103    1799
20180104    1751
20180105    1906
20180106    1853
20180107    1511
20180108    1700
20180109    1665
20180110    1765
20180111    1580
20180112    1800
20180113    1746
20180114    1525
20180115    1660
20180116    1636
20180117    1941
20180118    1881
20180119    1980
20180120    1870
20180121    1639
20180122    1637
20180123    1599
20180124    1620
20180125    1629
20180126    1700
20180127    1728
20180128    1470
20180129    1639
20180130    1653
            ... 
20181202    1445
20181203    1664
20181204    1827
20181205    1899
20181206    1916
20181207    2031
20181208    1693
20181209    1393
20181210    1735
20181211    1747
20181212    1855
20181213    1917
20181214    2097
20181215    1836
20181216    1468
20181217    1791
20181218    1738
20181219    1955
20181220    2124
20181221    2032
20181222    1962
20181223    1450
20181224    1842
20181225    1447
20181226    1874
20181227    1753
20181228    1747
2018122

In [57]:
busUser.shape

(361024, 10)

# To_csv

In [36]:
#df.to_csv("spot_information.csv", encoding="ANSI")